<a href="https://colab.research.google.com/github/anirudh201098/Store-Item-Demand-Forecasting/blob/master/LSTM_Autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import os
import datetime

In [ ]:
data=pd.read_csv('/content/drive/My Drive/Dataset/train.csv',parse_dates=['date'],index_col=['date'])

In [ ]:
data= data.sort_values('date', ascending=True)

In [ ]:
test=data[data.index.year==2017]
test.reset_index(level=0,inplace=True)
train=data[data.index.year!=2017]
train.reset_index(level=0,inplace=True)

In [ ]:
test.shape

(182500, 4)

In [ ]:
train.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-01,7,12,26
2,2013-01-01,7,46,27
3,2013-01-01,8,12,54
4,2013-01-01,9,12,35


In [ ]:

train_data = pd.DataFrame({'year': train['date'].dt.year-2013, 'month': train['date'].dt.month, 'day': train['date'].dt.day, 
                       'weekday': train['date'].dt.weekday,
                       'store': train.store, 'item': train.item,'sales':train.sales
                      }, columns = ['year', 'month', 'day', 'weekday', 'store', 'item','sales'],
                    )



test_data = pd.DataFrame({'year': test['date'].dt.year-2013, 'month': test['date'].dt.month, 'day': test['date'].dt.day, 
                       'weekday': test['date'].dt.weekday,
                       'store': test.store, 'item': test.item,'sales':test.sales
                      }, columns = ['year', 'month', 'day', 'weekday', 'store', 'item','sales'],
                    )


In [ ]:
X=np.array(train_data.drop('sales',axis=1))
y=np.array(train_data['sales'])
X_test=np.array(test_data.drop('sales',axis=1))
y_test=np.array(test_data['sales'])

In [ ]:
from sklearn.model_selection import train_test_split
def split(random,n,X_train,y_train,m):
  random_split =random
  train_ratio = n

  if random_split:
      X_train1, X_val, y_train1, y_val = train_test_split(X_train, y_train, test_size=(1-train_ratio), random_state=0, shuffle = True)
  else:
    if m==1:
        X_train1 = X_train[(X_train[:,0]!=n)|(X_train[:,1]>6)]
        y_train1 = y_train[(X_train[:,0]!=n)|(X_train[:,1]>6)]
        X_val = X_train[(X_train[:,0]==n)&(X_train[:,1]<=6)]
        y_val = y_train[(X_train[:,0]==n)&(X_train[:,1]<=6)]
    else: 
        X_train1 = X_train[(X_train[:,0]!=n)|(X_train[:,1]<6)]
        y_train1 = y_train[(X_train[:,0]!=n)|(X_train[:,1]<6)]
        X_val = X_train[(X_train[:,0]==n)&(X_train[:,1]>=6)]
        y_val = y_train[(X_train[:,0]==n)&(X_train[:,1]>=6)]

  return X_train1,y_train1,X_val,y_val

X_train,y_train,X_val,y_val=split(False,0,X,y,0)
print("training: ", X_train.shape, y_train.shape)
print("validation: ", X_val.shape, y_val.shape)

training:  (623500, 6) (623500,)
validation:  (107000, 6) (107000,)


In [ ]:
from keras.models import Model as KerasModel
from keras.layers import Input, Dense, Activation, Reshape, Dropout,LSTM,Flatten
from keras.layers import Concatenate,TimeDistributed,RepeatVector
from keras.layers.embeddings import Embedding
from keras import optimizers, regularizers
from keras.callbacks import EarlyStopping
import keras.backend as K
import keras
import tensorflow as tf


In [ ]:
def split_features(X):
    X_list = []
    for i in range(6):
        X_list.append(X[:,i])
    
    return X_list

# custom loss function
def smape(x, y):
    return 100.*K.mean(2*K.abs(x-y)/(K.abs(x)+K.abs(y)))


class LSTMwithEmbeddings():
    
    def __init__(self, X_train, y_train, X_val, y_val):
        super().__init__()
        self._build_model()
        #self.fit(X_train, y_train, X_val, y_val)
        
    def preprocess(self, X):
        X_list = split_features(X)
        return X_list
        
    def _build_model(self):
        ## year is a continuous feature
        inp_year = Input(shape=(1,), name="year")
        out_year = Embedding(5+1,3,name='year__embedding')(inp_year)
        out_year = Reshape(target_shape=(3,))(out_year)
        
        ## all other features are categorical and need embedding
        inp_month = Input(shape=(1,))
        out_month = Embedding(12+1, 7, name='month_embedding')(inp_month)
        out_month = Reshape(target_shape=(7,))(out_month)
        
        inp_day = Input(shape=(1,))
        out_day = Embedding(31+1, 16, name='day_embedding')(inp_day)
        out_day = Reshape(target_shape=(16,))(out_day)
        
        inp_weekday = Input(shape=(1,))
        out_weekday = Embedding(7+1, 4, name='weekday_embedding')(inp_weekday)
        out_weekday = Reshape(target_shape=(4,))(out_weekday)
        
        inp_stores = Input(shape=(1,))
        out_stores = Embedding(10+1, 6, name='stores_embedding')(inp_stores)
        out_stores = Reshape(target_shape=(6,))(out_stores)
        
        inp_items = Input(shape=(1,))
        out_items = Embedding(50+1, 26, name='items_embedding')(inp_items)
        out_items = Reshape(target_shape=(26,))(out_items)
        
        
        inp_model = [inp_year, inp_month, inp_day, inp_weekday, inp_stores, inp_items]
        out_embeddings = [out_year, out_month, out_day, out_weekday, out_stores, out_items]
        
        out_model = Concatenate()(out_embeddings)
        #out_model = Dense(1000)(out_model)
        #out_model = Activation('relu')(out_model)
        #out_model = Dropout(0.3)(out_model)
        out_model=Reshape(target_shape=(62,1))(out_model)
        out_model=LSTM(112,activation='relu',return_sequences=False)(out_model)
        #out_model=Reshape(target_shape=(62,112))(out_model)
        out_model=RepeatVector(1)(out_model)
        #out_model=Reshape(target_shape=(112,4))(out_model)
        out_model=LSTM(96,activation='relu',return_sequences=True)(out_model)
        #out_model=Reshape(target_shape=(62,1))(out_model)
        #out_model = Dense(64)(out_model)
        #out_model = Activation('relu')(out_model)
        out_model = TimeDistributed(Dense(32, kernel_initializer='glorot_normal'))(out_model)
        out_model = Activation('relu')(out_model)
        #out_model = Dropout(0.3)(out_model)
        out_model = TimeDistributed(Dense(1, kernel_initializer='glorot_normal'))(out_model)
        
        self.model = KerasModel(inputs=inp_model, outputs=out_model)
        
        self.model.compile(optimizer='Adam', metrics=['mean_absolute_error'],loss=[keras.losses.mean_squared_error])
        
    
    def fit(self, X_train, y_train, X_val, y_val):
        self.model.fit(self.preprocess(X_train), y_train,
                       validation_data=(self.preprocess(X_val), y_val),#5+2
                       epochs=2, batch_size=112,  #5+1+2+2
                       #callbacks=[EarlyStopping(monitor='val_loss', patience=2)],
                   )
        self.model.save_weights('/content/drive/My Drive/Dataset/weightsauto5.h5')
        #print("Result on validation data: ", self.evaluate(X_val, y_val))
    def load(self):
      self.model.load_weights('/content/drive/My Drive/Dataset/weightsauto5.h5')
    def prediction(self, X):
        return self.model.predict(self.preprocess(X)).flatten()

In [ ]:
model=LSTMwithEmbeddings(X_train, y_train, X_val, y_val)

ValueError: ignored

In [ ]:

#model.fit(X_train, y_train, X_val, y_val)
model.load()

In [ ]:
def fun(i,j):
    X_train,y_train,X_val,y_val=split(False,i,X,y,j)
    model.fit(X_train, y_train, X_val, y_val)
    model.load() 
    y_pred=model.prediction(X_test)
    return y_pred

In [ ]:
pred=fun(0,0)

Epoch 1/2
5567/5567 [==============================] - 545s 98ms/step - loss: 51.8078 - mean_absolute_error: 5.5402 - val_loss: 46.6297 - val_mean_absolute_error: 5.2700
Epoch 2/2
5567/5567 [==============================] - 544s 98ms/step - loss: 51.7561 - mean_absolute_error: 5.5370 - val_loss: 46.4266 - val_mean_absolute_error: 5.2562


In [ ]:
pred2=fun(0,1)

Epoch 1/2
5715/5715 [==============================] - 560s 98ms/step - loss: 52.2471 - mean_absolute_error: 5.5681 - val_loss: 42.2553 - val_mean_absolute_error: 4.9964
Epoch 2/2
5715/5715 [==============================] - 561s 98ms/step - loss: 52.2404 - mean_absolute_error: 5.5688 - val_loss: 42.1607 - val_mean_absolute_error: 4.9936


In [ ]:
pred3=fun(1,0)

Epoch 1/2
5567/5567 [==============================] - 542s 97ms/step - loss: 50.5407 - mean_absolute_error: 5.4714 - val_loss: 53.6769 - val_mean_absolute_error: 5.6480
Epoch 2/2
5567/5567 [==============================] - 546s 98ms/step - loss: 50.5214 - mean_absolute_error: 5.4704 - val_loss: 53.6055 - val_mean_absolute_error: 5.6496


In [ ]:
pred4=fun(1,1)

Epoch 1/2
5715/5715 [==============================] - 557s 98ms/step - loss: 51.3737 - mean_absolute_error: 5.5183 - val_loss: 48.1415 - val_mean_absolute_error: 5.3400
Epoch 2/2
5715/5715 [==============================] - 554s 97ms/step - loss: 51.3837 - mean_absolute_error: 5.5191 - val_loss: 48.3143 - val_mean_absolute_error: 5.3420


In [ ]:
pred5=fun(2,0)

Epoch 1/2
5567/5567 [==============================] - 543s 98ms/step - loss: 50.1351 - mean_absolute_error: 5.4509 - val_loss: 56.4828 - val_mean_absolute_error: 5.7791
Epoch 2/2
5567/5567 [==============================] - 540s 97ms/step - loss: 50.1220 - mean_absolute_error: 5.4499 - val_loss: 55.9568 - val_mean_absolute_error: 5.7709


In [ ]:
pred6=fun(2,1)

Epoch 1/2
5715/5715 [==============================] - 558s 98ms/step - loss: 51.1236 - mean_absolute_error: 5.5053 - val_loss: 49.8468 - val_mean_absolute_error: 5.4279
Epoch 2/2
5715/5715 [==============================] - 557s 97ms/step - loss: 51.1255 - mean_absolute_error: 5.5059 - val_loss: 50.5634 - val_mean_absolute_error: 5.4913


In [ ]:
pred7=fun(3,0)

Epoch 1/2
5567/5567 [==============================] - 541s 97ms/step - loss: 49.2663 - mean_absolute_error: 5.4046 - val_loss: 60.8332 - val_mean_absolute_error: 6.0283
Epoch 2/2
5567/5567 [==============================] - 543s 98ms/step - loss: 49.2537 - mean_absolute_error: 5.4038 - val_loss: 61.2496 - val_mean_absolute_error: 6.0645


In [ ]:
y_pred=fun(3,1)

Epoch 1/2
5710/5710 [==============================] - 553s 97ms/step - loss: 50.4990 - mean_absolute_error: 5.4680 - val_loss: 54.3725 - val_mean_absolute_error: 5.6858
Epoch 2/2
5710/5710 [==============================] - 550s 96ms/step - loss: 50.4675 - mean_absolute_error: 5.4670 - val_loss: 54.2227 - val_mean_absolute_error: 5.6836


In [ ]:
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
from keras.metrics import mean_absolute_percentage_error


In [ ]:
y_pred=model.prediction(X_test)

In [ ]:
print(y_pred.shape)
y_test[:17]

(182500,)


array([18, 36, 38, 43, 66, 53, 24, 81, 49, 30, 65, 26, 33, 22, 53, 70, 84],
      dtype=int64)

In [ ]:
y_pred[:17]

array([19.8163  , 34.04424 , 45.605022, 41.765697, 59.655777, 39.204147,
       22.949257, 78.99566 , 45.589333, 24.377762, 65.125145, 23.591564,
       31.140583, 18.913414, 49.587605, 73.34438 , 69.32466 ],
      dtype=float32)

In [ ]:
r2_score(y_test,y_pred)

0.9300370491269196

In [ ]:
mean_absolute_percentage_error(y_test,y_pred)

<tf.Tensor: shape=(), dtype=float32, numpy=12.267833>

In [ ]:
mean_absolute_error(y_test,y_pred)

6.357305220643135

In [ ]:
import math
math.sqrt(mean_squared_error(y_test,y_pred))

8.345826697737627

In [ ]:
SS_Residual = sum((y_test-y_pred)**2)  
print("SSE:",SS_Residual)     
SS_Total = sum((y_test-np.mean(y_test))**2)
print("TSS:",SS_Total)     
r_squared = 1 - (float(SS_Residual))/SS_Total
print("R-Square",r_squared)
adjusted_r_squared = 1 - (1-r_squared)*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
print("Adjusted R-square",adjusted_r_squared)

SSE: 12711640.246534308
TSS: 181691024.8624723
R-Square 0.9300370491269112
Adjusted R-square 0.9300347488868733


In [ ]:
for i in range(1,13):
  X_test1=X_test[X_test[:,2]==i]
  y_test1=y_test[X_test[:,2]==i]
  y_pred1=model.prediction(X_test1)
  print(i,r2_score(y_test1,y_pred1))

1 0.9283117955519771
2 0.9383807952888419
3 0.9203974028057573
4 0.9313028564108558
5 0.9297803230207687
6 0.9321943771698717
7 0.928067139557482
8 0.9268229394192204
9 0.935787073068902
10 0.9271643298876088
11 0.931443066907179
12 0.9273492119985556
